* 前馈神经网络、网络层数、输入层、隐藏层、输出层、隐藏单元、激活函数的概念。
* 感知机相关；定义简单的几层网络（激活函数sigmoid），递归使用链式法则来实现反向传播。
* 激活函数的种类以及各自的提出背景、优缺点。（和线性模型对比，线性模型的局限性，去线性化）
* 深度学习中的正则化（参数范数惩罚：L1正则化、L2正则化；数据集增强；噪声添加；early stop；Dropout层）、正则化的介绍。
* 深度模型中的优化：参数初始化策略；自适应学习率算法（梯度下降、AdaGrad、RMSProp、Adam；优化算法的选择）；batch norm层（提出背景、解决什么问题、层在训练和测试阶段的计算公式）；layer norm层。
* fastText在保持分类效果的同时，大大缩短了训练时间
* fastText适合大型数据+高效的训练速度，支持多语言表达，专注于文本分类，在许多标准问题上实现当下最好的表现（例如文本倾向性分析或标签预测）

### FastText的原理
#### fastText方法包含三个部分：模型架构，层次Softmax和N-gram特征
fastText在预测标签时使用了非线性激活函数，但在中间层不使用非线性激活函数。<br>
##### 第一部分：fastText模型架构：
其中x1,x2,...,xN−1,xN表示一个文本中的n-gram向量，每个特征是词向量的平均值。这和前文中提到的cbow相似，cbow用上下文去预测中心词，而此处用全部的n-gram去预测指定类别。
##### 第二部分 层次SoftMax
对于有大量类别的数据集，fastText使用了一个分层分类器（而非扁平式架构）。不同的类别被整合进树形结构中（想象下二叉树而非 list）。在某些文本分类任务中类别很多，计算线性分类器的复杂度高。为了改善运行时间，fastText 模型使用了层次 Softmax 技巧。层次 Softmax 技巧建立在哈弗曼编码的基础上，对标签进行编码，能够极大地缩小模型预测目标的数量。<br>

fastText 也利用了类别（class）不均衡这个事实（一些类别出现次数比其他的更多），通过使用 Huffman 算法建立用于表征类别的树形结构。因此，频繁出现类别的树形结构的深度要比不频繁出现类别的树形结构的深度要小，这也使得进一步的计算效率更高。
##### 第三部分：N-gram子词特征
fastText 可以用于文本分类和句子分类。不管是文本分类还是句子分类，我们常用的特征是词袋模型。但词袋模型不能考虑词之间的顺序，因此 fastText 还加入了 N-gram 特征。在 fasttext 中，每个词被看做是 n-gram字母串包。为了区分前后缀情况，"<"， ">"符号被加到了词的前后端。除了词的子串外，词本身也被包含进了 n-gram字母串包。以 where 为例，n=3 的情况下，其子串分别为<wh, whe, her, ere, re>，以及其本身 。

### fastText和word2vec的区别
* 相似处：<br>
1.图模型结构很像，都是采用embedding向量的形式，得到word的隐向量表达。<br>
2.都采用很多相似的优化方法，比如使用Hierarchical softmax优化训练和预测中的打分速度。
* 不同处：<br>
1.模型的输出层：word2vec的输出层，对应的是每一个term，计算某term的概率最大；而fasttext的输出层对应的是分类的label。不过不管输出层对应的是什么内容，起对应的vector都不会被保留和使用。<br>
2.模型的输入层：word2vec的输出层，是 context window 内的term；而fasttext 对应的整个sentence的内容，包括term，也包括 n-gram的内容。
* 两者本质的不同，体现在 h-softmax的使用：<br>
1.Word2vec的目的是得到词向量，该词向量 最终是在输入层得到，输出层对应的 h-softmax也会生成一系列的向量，但最终都被抛弃，不会使用。<br>
2.fastText则充分利用了h-softmax的分类功能，遍历分类树的所有叶节点，找到概率最大的label（一个或者N个）